In [1]:
# Concurency with Futures.

In [3]:
# Usually programs are executed sequentially - one task at a time.
# However it is possible to execute many task at one moment. Actually you are still executing
# still one task ot a time, but when the CPU is waiting for a response from an external resource
# it is performing another task.

In [4]:
# concurrent.futures is a module that allows you to create this virtulization of parallel tasks.

In [5]:
# Let's see an example:
# We will do the same thing as in the last lecture but will try to make it concurent.

In [6]:
# Firstly let's try load our list of ids.

In [7]:
import json
albums_id = json.load(open('albums_ids.json', 'r'))

In [8]:
albums_id

['6VH2op0GKIl3WNTbZmmcmI',
 '23FJTTzUIUjhmimOE2CTX2',
 '0kTe1sQd9yhDsdG2Zth7X6',
 '4wExFfncaUIqSgoxnqa3Eh',
 '3otvl9PN3kOgk5uwAh1CBL',
 '56G9UnPmRifbubzPDJfAyd',
 '228mANuRrV20jS5DCA0eER',
 '5WRmchevXvk5eXPT0gEuTL',
 '1jCYuXr0Ujke24z1ymBr5U',
 '1W5CtQ7Ng0kP3lXyz7PIT2',
 '4wVHvawxZy52Oemd4sGyUm',
 '0ui4S0TZghkf1d1Wz0oWpk',
 '6vSiY2OVanKKforfEOT2PD',
 '3uhD8hNpb0m3iIZ18RHH5u',
 '3xiPkaTzfC48CcsXYgz5v0',
 '1lZahjeu4AhPkg9JARZr5F',
 '0ovKDDAHiTwg4AEjKdgdWo',
 '4Q7cPyiP8cMIlUEHAqeYfd',
 '26tH0kjUhkxBEd3ipGkx3Y',
 '7gS8ozSkvPW3VBPLnXOZ7S']

In [11]:
# Now as the last time we must autentificate ourselves on spotify.
import requests

In [77]:
# Now we need to setup the CLIENT_ID and the CLIENT_SECRET
CLIENT_ID = '1e8df497a8254e4b8ee14e464f164d71'
CLIENT_SECRET = 'bfaa9a12282640d1b38c2f145ede3ef5'

In [78]:
# To get the data from spotify we must firstly autetithicate ourselves, let's define the 
# Autehtification url
AUTH_URL = 'https://accounts.spotify.com/api/token'

In [79]:
# Now we can send the aiutetification request
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

In [80]:
# Now we must use the json funtion to convert the response into json format
auth_response_data = auth_response.json()

In [81]:
# Now we can get the accese token to get the data
access_token = auth_response_data['access_token']

In [82]:
# Let's extract the headers
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [83]:
# Now we must create a function that will request the tracks and will add it to a global variable

In [84]:
TRACKS_NAMES = []
BASE_URL = 'https://api.spotify.com/v1/'

In [87]:
def get_tracks(album_id):
    r = requests.get(BASE_URL + 'albums/' + album_id + '/tracks', 
                headers=headers)
    print(r.json())
    data = r.json()['items']
    return [track['name'] for track in data]

In [61]:
# Now we can import the modules for concurency

In [62]:
from concurrent import futures

In [63]:
# let't set the maximal number and the real number of worker threads. To not create unnecessary
# threads.
MAX_WORKERS = 20
workers = min(MAX_WORKERS, len(albums_id))

In [64]:
# Now we can start the conciurent task
with futures.ThreadPoolExecutor(workers) as executor:
    to_do = []
    for album_id in sorted(albums_id):
        future = executor.submit(get_tracks, album_id)
        to_do.append(future)
    results = []
    for future in futures.as_completed(to_do):
        res = future.result()
        TRACKS_NAMES.extend(res)

In [65]:
TRACKS_NAMES

['Good Times Bad Times - Live: O2 Arena, London - December 10, 2007',
 'Ramble On - Live: O2 Arena, London - December 10, 2007',
 'Black Dog - Live: O2 Arena, London - December 10, 2007',
 'In My Time of Dying - Live: O2 Arena, London - December 10, 2007',
 'For Your Life - Live: O2 Arena, London - December 10, 2007',
 'Trampled Under Foot - Live: O2 Arena, London - December 10, 2007',
 "Nobody's Fault but Mine - Live: O2 Arena, London - December 10, 2007",
 'No Quarter - Live: O2 Arena, London - December 10, 2007',
 "Since I've Been Loving You - Live: O2 Arena, London - December 10, 2007",
 'Dazed and Confused - Live: O2 Arena, London - December 10, 2007',
 'Stairway to Heaven - Live: O2 Arena, London - December 10, 2007',
 'The Song Remains the Same - Live: O2 Arena, London - December 10, 2007',
 'Misty Mountain Hop - Live: O2 Arena, London - December 10, 2007',
 'Kashmir - Live: O2 Arena, London - December 10, 2007',
 'Whole Lotta Love - Live: O2 Arena, London - December 10, 2007',


In [72]:
TRACKS_NAMES = []

In [90]:
# Now let's see how much it takes

In [93]:
%%timeit -n 1 -r 1
with futures.ThreadPoolExecutor(workers) as executor:
    to_do = []
    for album_id in sorted(albums_id):
        future = executor.submit(get_tracks, album_id)
        to_do.append(future)
    results = []
    for future in futures.as_completed(to_do):
        res = future.result()
        TRACKS_NAMES.extend(res)

{'href': 'https://api.spotify.com/v1/albums/1lZahjeu4AhPkg9JARZr5F/tracks?offset=0&limit=20', 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/36QJpDe2go2KgaRleHCDTp'}, 'href': 'https://api.spotify.com/v1/artists/36QJpDe2go2KgaRleHCDTp', 'id': '36QJpDe2go2KgaRleHCDTp', 'name': 'Led Zeppelin', 'type': 'artist', 'uri': 'spotify:artist:36QJpDe2go2KgaRleHCDTp'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SI', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'UA', 'US', 'UY', 'VN', 'XK', 'ZA'], 'disc_number': 1, 'duration_ms': 25386

{'href': 'https://api.spotify.com/v1/albums/228mANuRrV20jS5DCA0eER/tracks?offset=0&limit=20', 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/36QJpDe2go2KgaRleHCDTp'}, 'href': 'https://api.spotify.com/v1/artists/36QJpDe2go2KgaRleHCDTp', 'id': '36QJpDe2go2KgaRleHCDTp', 'name': 'Led Zeppelin', 'type': 'artist', 'uri': 'spotify:artist:36QJpDe2go2KgaRleHCDTp'}], 'available_markets': ['AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'EC', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IT', 'JP', 'KZ', 'LT', 'LU', 'LV', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'PA', 'PE', 'PH', 'PL', 'PT', 'PY', 'RO', 'RS', 'RU', 'SE', 'SG', 'SI', 'SK', 'SV', 'TH', 'TR', 'TW', 'UA', 'US', 'UY', 'VN', 'XK', 'ZA'], 'disc_number': 1, 'duration_ms': 158413, 'explicit': False, 'external_urls': {'spotify': 'https://open.spotify.com/track/50B4cVW

In [94]:
# As you see it does the same task faster.